In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def load_checkpoint(model, checkpoint_path):
    model_ckp = torch.load(checkpoint_path)
    model.load_state_dict(model_ckp['model_state_dict'])
    return model

class NewTrainManager(object):
    def __init__(self, student, teacher=None, young=None, train_loader=None, test_loader=None, train_config={}):
        self.student = student
        self.teacher = teacher
        self.young = young
 
        self.have_teacher = bool(self.teacher)
        self.have_young = bool(self.young)
 
        self.device = train_config['device'] #cuda or cpu
        self.name = train_config['name'] #current training model's name
        self.optimizer = optim.SGD(self.student.parameters(),
                                   lr=train_config['learning_rate'],
                                   momentum=train_config['momentum'],
                                   weight_decay=train_config['weight_decay'])
        if self.have_teacher:
            self.teacher.eval()
            self.teacher.train(mode=False)
 
        if self.have_young:
            self.young.eval()
            self.young.train(mode=False)
            
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.config = train_config
 
        self.best_epoch = 0
        self.best_accuracy = 0
    
    def train(self):
        lambda_ = self.config['lambda_student']
        gamma_ = self.config['gamma']
        T = self.config['T_student']
 
        epochs = self.config['epochs']
        trial_id = self.config['trial_id']
        
        max_val_acc = 0
        iteration = 0
        best_acc = 0
        best_epo = 0
        criterion = nn.CrossEntropyLoss()
        
        training_time = time.time()
        #print("여기1")
        for epoch in range(1, epochs+1):
            start_time = time.time()
            self.student.train()
            self.adjust_learning_rate(self.optimizer, epoch)
            loss = 0
            #print("여기2")
            for batch_idx, (data, target) in enumerate(self.train_loader):
                iteration += 1
                data = data.to(self.device)
                target = target.to(self.device)
                self.optimizer.zero_grad()
                output = self.student(data)
                # Standard Learning Loss ( Classification Loss)
                loss_SL = criterion(output, target) 
                loss = loss_SL

                #print("여기3")

                if self.have_teacher:
                    teacher_outputs = self.teacher(data)
                    # Knowledge Distillation Loss
                    loss_KD = nn.KLDivLoss()(F.log_softmax(output / T, dim=1),
                                                      F.softmax(teacher_outputs / T, dim=1))
                    loss = (1 - lambda_) * loss_SL + lambda_ * T * T * loss_KD
                    #print("여기4")

                    if self.have_young:
                        young_outputs = self.young(data)
                        # ReverseKD from younger to older
                        loss_ReKD = nn.KLDivLoss()(F.log_softmax(output / T, dim=1),
                                                      F.softmax(young_outputs / T, dim = 1))
                        loss = (1 - lambda_ - gamma_) * loss_SL + lambda_ * T * T * loss_KD + gamma_ * T * T * loss_ReKD
                    
                loss.backward()
                self.optimizer.step()
            
            print("epoch {0}/{1} --- {2:0.3f} seconds --- ".format(epoch, epochs, time.time() - start_time))
            val_acc = self.validate(step=epoch)
            if val_acc > best_acc:
                best_acc = val_acc
                best_epoch = epoch
                self.save(epoch, name='{}_{}_{}_{}_{}_best.pth.tar'.format(self.name, trial_id, T, lambda_, gamma_))
        save_model = create_cnn_model(self.name, 'cifar10', use_cuda=True)
        save_model = load_checkpoint(save_model, os.path.join('./','{}_{}_{}_{}_{}_best.pth.tar'.format(self.name, trial_id, T, lambda_, gamma_)))
        torch.save({
                'model_state_dict': save_model.state_dict(),
                # 'optimizer_state_dict': save_model.optimizer.state_dict(),
                'epoch': best_epoch,
            }, '{}_{}_{}_{}_{}_{}_best.pth.tar'.format(best_acc,self.name, trial_id, T, lambda_, gamma_))
        print("Total elapsed time : {0:0.3f} seconds --- ".format(time.time() - training_time))
        print("Best accuracy was : {}".format(best_acc))
        return best_acc
    
    def validate(self, step=0):
        self.student.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            acc = 0
            for images, labels in self.test_loader:
                images = images.to(self.device)
                labels = labels.to(self.device)
                outputs = self.student(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            # self.accuracy_history.append(acc)
            acc = 100 * correct / total
            
            print('{{"metric": "{}_val_accuracy", "value": {}}}'.format(self.name, acc))
            return acc
    
    def save(self, epoch, name=None):
        trial_id = self.config['trial_id']
        if name is None:
            torch.save({
                'epoch': epoch,
                'model_state_dict': self.student.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
            }, '{}_{}_epoch{}.pth.tar'.format(self.name, trial_id, epoch))
        else:
            torch.save({
                'model_state_dict': self.student.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'epoch': epoch,
            }, name)
    
    def adjust_learning_rate(self, optimizer, epoch):
        epochs = self.config['epochs']
        models_are_plane = self.config['is_plane']
        
        # depending on dataset
        if models_are_plane:
            lr = 0.01
        else:
            if epoch < int(epoch/2.0):
                lr = 0.1
            elif epoch < int(epochs*3/4.0):
                lr = 0.1 * 0.1
            else:
                lr = 0.1 * 0.01
        
        # update optimizer's learning rate
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/DLF/student')
import copy
import math
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as torch_models
import torch.nn.functional as F
import torch.optim as optim
import time
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/DLF/TaKD')

import data_loader
import model_factory
import plain_cnn_cifar
import resnet_cifar

from data_loader import get_cifar
from model_factory import create_cnn_model, is_resnet

from resnet_cifar import *
from plain_cnn_cifar import *

In [0]:
myname = "CH"
batch_size = 128

dataset = 'cifar10'
num_classes = 10 # 10 or 100
train_loader, test_loader = get_cifar(num_classes, dataset_dir='/content/drive/My Drive/Colab Notebooks/DLF/cifar/crop', batch_size=batch_size, crop = True)

Files already downloaded and verified
Files already downloaded and verified


# Making student

In [0]:
# teacher_checkpoint = '/content/drive/My Drive/Colab Notebooks/DLF/teacher/' + str(teacher_name)

teacher = 'resnet26' # input teacher's model
teacher_checkpoint =  '/content/drive/My Drive/Colab Notebooks/DLF/teacher/resnet26_paik_20_128 # 4_best.pth.tar'
# copy and paste teacher's path

teacher_model = create_cnn_model(teacher, dataset, use_cuda=True)
print("---------- Loading Teacher -------")
teacher_model = load_checkpoint(teacher_model, teacher_checkpoint)
print("대여")

---------- Loading Teacher -------
대여


In [0]:
epochs = 80
lambda_ = 0.1
gamma_ = 0.05
trial_num = 1
chseed = 20 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

student = 'resnet8'
student_model = create_cnn_model(student, dataset, use_cuda=True)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': student,
    'T_student': 3, #config.get('T_student')
		'lambda_student': lambda_, #config.get('lambda_student')
		'gamma' : gamma_ # 
	}
  
young = ''
young_checkpoint = ''
# copy and paste

if young:
    print("---------- Loading Young -------")
    young_model = create_cnn_model(young, dataset, use_cuda=True)
    young_model = load_checkpoint(young_model, young_checkpoint)
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = young_model, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    student_trainer.train()
else:
    print("---------- Without Young -------")
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    print(1)
    student_trainer.train() 

---------- Without Young -------
1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


epoch 1/80 --- 48.509 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 38.01}
epoch 2/80 --- 48.380 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 46.11}
epoch 3/80 --- 48.316 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 50.21}
epoch 4/80 --- 48.303 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 53.6}
epoch 5/80 --- 48.362 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 58.12}
epoch 6/80 --- 48.342 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 61.86}
epoch 7/80 --- 48.355 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 61.2}
epoch 8/80 --- 48.368 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 60.13}
epoch 9/80 --- 48.328 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 63.4}
epoch 10/80 --- 48.358 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 66.97}
epoch 11/80 --- 48.346 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 68.25}
epoch 12/80 --- 48.338 seconds --- 
{"metric": "resnet8

In [0]:
epochs = 80
lambda_ = 0.1
gamma_ = 0.8
trial_num = 2
chseed = 20 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

student = 'resnet8'
student_model = create_cnn_model(student, dataset, use_cuda=True)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': student,
    'T_student': 3, #config.get('T_student')
		'lambda_student': lambda_, #config.get('lambda_student')
		'gamma' : gamma_ # 
	}

young = ''
young_checkpoint = ''
# copy and paste

if young:
    print("---------- Loading Young -------")
    young_model = create_cnn_model(young, dataset, use_cuda=True)
    young_model = load_checkpoint(young_model, young_checkpoint)
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = young_model, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    student_trainer.train()
else:
    print("---------- Without Young -------")
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    print(1)
    student_trainer.train() 

---------- Without Young -------
1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


epoch 1/80 --- 48.299 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 38.16}
epoch 2/80 --- 48.154 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 46.47}
epoch 3/80 --- 48.190 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 52.6}
epoch 4/80 --- 48.181 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 54.21}
epoch 5/80 --- 48.179 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 56.57}
epoch 6/80 --- 48.197 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 63.04}
epoch 7/80 --- 48.185 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 61.55}
epoch 8/80 --- 48.134 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 61.94}
epoch 9/80 --- 48.181 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 64.23}
epoch 10/80 --- 48.155 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 65.3}
epoch 11/80 --- 48.139 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 67.42}
epoch 12/80 --- 48.202 seconds --- 
{"metric": "resnet

In [0]:
epochs = 80
lambda_ = 0.4
gamma_ = 0.1
trial_num = 3
chseed = 20 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

student = 'resnet8'
student_model = create_cnn_model(student, dataset, use_cuda=True)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': student,
    'T_student': 3, #config.get('T_student')
		'lambda_student': lambda_, #config.get('lambda_student')
		'gamma' : gamma_ # 
	}

young = ''
young_checkpoint = ''
# copy and paste

if young:
    print("---------- Loading Young -------")
    young_model = create_cnn_model(young, dataset, use_cuda=True)
    young_model = load_checkpoint(young_model, young_checkpoint)
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = young_model, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    student_trainer.train()
else:
    print("---------- Without Young -------")
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    print(1)
    student_trainer.train() 

---------- Without Young -------
1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


epoch 1/80 --- 48.167 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 36.83}
epoch 2/80 --- 47.762 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 43.64}
epoch 3/80 --- 47.904 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 49.46}
epoch 4/80 --- 47.938 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 52.99}
epoch 5/80 --- 47.927 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 54.31}
epoch 6/80 --- 47.902 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 59.92}
epoch 7/80 --- 47.972 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 59.52}
epoch 8/80 --- 47.933 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 62.07}
epoch 9/80 --- 47.941 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 65.31}
epoch 10/80 --- 47.941 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 66.46}
epoch 11/80 --- 47.942 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 65.53}
epoch 12/80 --- 47.933 seconds --- 
{"metric": "resn

In [0]:
epochs = 80
lambda_ = 0.4
gamma_ = 0.4
trial_num = 4
chseed = 20 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

student = 'resnet8'
student_model = create_cnn_model(student, dataset, use_cuda=True)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': student,
    'T_student': 3, #config.get('T_student')
		'lambda_student': lambda_, #config.get('lambda_student')
		'gamma' : gamma_ # 
	}

young = ''
young_checkpoint = ''
# copy and paste

if young:
    print("---------- Loading Young -------")
    young_model = create_cnn_model(young, dataset, use_cuda=True)
    young_model = load_checkpoint(young_model, young_checkpoint)
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = young_model, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    student_trainer.train()
else:
    print("---------- Without Young -------")
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    print(1)
    student_trainer.train() 

---------- Without Young -------
1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


epoch 1/80 --- 48.529 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 37.24}
epoch 2/80 --- 48.238 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 43.55}
epoch 3/80 --- 48.261 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 48.31}
epoch 4/80 --- 48.186 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 52.85}
epoch 5/80 --- 48.273 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 55.39}
epoch 6/80 --- 48.200 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 58.57}
epoch 7/80 --- 48.197 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 58.09}
epoch 8/80 --- 48.185 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 61.8}
epoch 9/80 --- 48.209 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 63.54}
epoch 10/80 --- 48.225 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 66.54}
epoch 11/80 --- 48.363 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 67.36}
epoch 12/80 --- 48.212 seconds --- 
{"metric": "resne

In [0]:
epochs = 80
lambda_ = 0.8
gamma_ = 0.1
trial_num = 5
chseed = 20 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

student = 'resnet8'
student_model = create_cnn_model(student, dataset, use_cuda=True)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': student,
    'T_student': 3, #config.get('T_student')
		'lambda_student': lambda_, #config.get('lambda_student')
		'gamma' : gamma_ # 
	}

young = ''
young_checkpoint = ''
# copy and paste

if young:
    print("---------- Loading Young -------")
    young_model = create_cnn_model(young, dataset, use_cuda=True)
    young_model = load_checkpoint(young_model, young_checkpoint)
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = young_model, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    student_trainer.train()
else:
    print("---------- Without Young -------")
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    print(1)
    student_trainer.train() 

---------- Without Young -------
1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


epoch 1/80 --- 47.350 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 32.85}
epoch 2/80 --- 46.948 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 38.13}
epoch 3/80 --- 46.935 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 43.16}
epoch 4/80 --- 46.887 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 47.93}
epoch 5/80 --- 46.849 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 53.52}
epoch 6/80 --- 46.888 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 51.46}
epoch 7/80 --- 46.930 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 57.45}
epoch 8/80 --- 46.895 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 59.17}
epoch 9/80 --- 46.911 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 61.53}
epoch 10/80 --- 46.959 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 62.89}
epoch 11/80 --- 46.952 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 63.74}
epoch 12/80 --- 46.899 seconds --- 
{"metric": "resn

In [0]:
epochs = 80
lambda_ = 0.9
gamma_ = 0.05
trial_num = 6
chseed = 20 # 20 31 55
torch.manual_seed(chseed); torch.cuda.manual_seed(chseed)
trial_id = myname + "_" + str(chseed) +"_" + str(batch_size) + " # " + str(trial_num)

student = 'resnet8'
student_model = create_cnn_model(student, dataset, use_cuda=True)

train_config = {
		'epochs': epochs,
		'learning_rate': 0.1,
		'momentum': 0.9,
		'weight_decay': 0.0001,
		'device': 'cuda',
		'is_plane': False,
		'trial_id': trial_id,
		'name': student,
    'T_student': 3, #config.get('T_student')
		'lambda_student': lambda_, #config.get('lambda_student')
		'gamma' : gamma_ # 
	}

young = ''
young_checkpoint = ''
# copy and paste

if young:
    print("---------- Loading Young -------")
    young_model = create_cnn_model(young, dataset, use_cuda=True)
    young_model = load_checkpoint(young_model, young_checkpoint)
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = young_model, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    student_trainer.train()
else:
    print("---------- Without Young -------")
    student_trainer = NewTrainManager(student_model, teacher=teacher_model, young = None, train_loader=train_loader, test_loader=test_loader, train_config=train_config)
    print(1)
    student_trainer.train() 

---------- Without Young -------
1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1946: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


epoch 1/80 --- 46.560 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 30.94}
epoch 2/80 --- 46.635 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 37.35}
epoch 3/80 --- 46.657 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 40.95}
epoch 4/80 --- 46.693 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 44.43}
epoch 5/80 --- 46.615 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 48.65}
epoch 6/80 --- 46.590 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 49.93}
epoch 7/80 --- 46.519 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 52.9}
epoch 8/80 --- 46.518 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 60.1}
epoch 9/80 --- 46.566 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 61.02}
epoch 10/80 --- 46.588 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 61.91}
epoch 11/80 --- 46.652 seconds --- 
{"metric": "resnet8_val_accuracy", "value": 61.73}
epoch 12/80 --- 46.661 seconds --- 
{"metric": "resnet